# Hw1: Developing an Information Retrieval System with Advanced Boolean Search

## About
This project aims to develop an Information Retrieval (IR) system that supports both standard Boolean queries and
proximity queries. The system is designed to handle a collection of text documents, building an Inverted Index and a
Positional Index to facilitate efficient document retrieval.

## importing libraries
Let's first import libraries we might need. we will use some libraries as typing checking, and some libraries like string just for punctuation.
and also, as it is said in Problem file, we can use nltk library for tokenizing. I implement two way for tokenizing, one using nltk and one without.

In [1]:
import os
import string
import re
from typing import List

import nltk


## Main classes
Now let's define our classes. I define two classes for this project. Token and Information Retrieval System. I use Token class inside of IR System.
### Algorithm
Befor going to implementation, let's summarize our algorithm and approach:
First of all, we get a list of documents in format of string; we need to tokenize all these documents. we can split all words using whitespace. Then, we need to get rid of punctuation. This will be done using nltk. at last, it's better to lower case all words so we can index for example "another" and "Another" in same. After tokenizing, we have to create our posting_list, which is the core of this project. To do this, we should loop over all documents, then inside this loop, we loop over all the tokens that are in the current document. then we check if the length of posting_list is zero, then we add this token as first word. else if the length of posting_list is more than 0, we find the correct index of the token in posting_list alphabetically. then we check if this token, has been already in posting_list, we just add the current document index in tokens.docs, else, we add this token in the posting_list, then add the current document index. Also we store index of occurence of the token in the relevant documents.
Now we create our posting list, we can easily query. for intersecting(AND) 2 words, we just need to return document's index of our 2 words that are occurred in same documents. for union(OR), we gather index of word1 with word2. for NOT, we just need to return all document's index that are not in the list of documents that contain our word. And at last, for near, we just need to find documents that either each of these word has been occurred near by at most k words on left or right. we have to cases:
 * 1)the first_word occurred before second_word.
 * 2)second_word occurred before first_word.

so we loop over a tuple (p1, p2) to cover ordered occurrence of each word. then for each word we loop over its documents, and in each document, we loop over index of that the token is occurred. then if other word(let's say second) is in documents[word1_idx:word1_idx+length], we add the current document index.

### Token class:

In [2]:
class Token:
    """
    In this class, I implement a token object, which store the valuse of a token(str), the list of documents that it is
    occurred in, and its index in them.
    ...
    Attributes:
    ----------
    word: str
        value of a token
    docs: list[dict]
        list of documents that it is occurred in, and its index in them. each document is represented as a dict with two
        key: 1) doc_idx: index of the documents
            2)indexes: list of index of the occurrence of the token in this document
    """

    def __init__(self, word: str):
        self.word: str = word
        self.docs: list[dict] = []

    def __str__(self):
        return self.word

    def __repr__(self):
        return self.word


## Preprocessing
Let's define our preprocess functions

In [3]:
stop_words = set(nltk.corpus.stopwords.words('english'))


def read_documents(doc_folder):
    """
    Reads all text files in the specified directory and returns a dictionary
    with document ids as keys and the content as values.
    """
    documents = {}
    for doc_file in os.listdir(doc_folder):
        if doc_file.endswith('.txt'):
            doc_path = os.path.join(doc_folder, doc_file)
            with open(doc_path, 'r', encoding='utf-8') as file:
                documents[doc_file] = file.read()
    return documents


def preprocess_text(text):
    """
    Preprocesses the text by tokenizing, converting to lowercase, removing punctuation,
    and filtering out stop words.
    """
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize text
    tokens = nltk.tokenize.word_tokenize(text)
    # Filter out stop words
    filtered_tokens = [token for token in tokens if token not in stop_words]
    return filtered_tokens


def preprocess_documents(documents):
    """
    Apply text preprocessing to each document in the dictionary.
    """
    preprocessed_docs = []
    for doc_id, content in enumerate(documents):
        preprocessed_docs.append(preprocess_text(content))
    return preprocessed_docs


## Indexing
Now let's define our main class:

In [4]:
class Token:
    def __init__(self, word: str):
        self.word = word
        self.docs = []

    def __str__(self):
        return self.word

    def __repr__(self):
        return self.word


class InvertedIndex:
    """
    In this class, I implement an information retrieval system which can search a query among documents.
    ...
    Attributes:
    -----------
    documents: List
        list of documents in format of string.
    posting_list: List[Token]
        list of Token objects. Tokens store a string and document's indexes
    stop_word: set
        set of stop words to check when tokenizing
    case_sensitive: bool
        a boolean to determine whether we want to distinguish between lowercase and uppercase form.

    Methods
    -------
    Methods defined here:
        __init__(self, documents: List, case_sensitive=False):
            Constructor will set initial attributes like documents and case_sensitive. NOTE that documents should be
            list of strings at first.
            :parameter
            ---------
            documents:List
                list of strings at first. but then chagnes to list of lists of strings
            :return
                None
        add_document(self, doc_idx, doc):
            this function will add a document to the posting list. it will loop over all tokens in the document and
            add them to the posting list.
            :param doc_idx:
                index of the document in the documents list
            :param doc:
                list of tokens in the document
            :return:
                None
        create_posting_list(self):
            calling this function, will create posting list of all occurred words cross all documents.
            :parameter
                None
            :return
                None
        get_token_index(self, x):
            this function find index of a word in posting list using binary search algorithm.
            :parameter
                x:str
                    the word you want to find its index
            :return
                int: index of the word in posting_list
        get_token(self, token):
                This function will return the token object that contains docs informations. if the given token is not in the
                posting_list, it return the spell corrected token.
                :param token:
                    token you want to fetch it from posting list
                :return:
                    return the instance of token from posting list
    """

    def __init__(self, documents: List, case_sensitive=False):
        """
        Constructor will set initial attributes like documents and case_sensitive. NOTE that documents should be
            list of strings at first.
            :parameter
            ---------
            documents:List
                list of strings at first. but then chagnes to list of lists of strings
            :return
                None
        """
        self.documents = documents
        self.posting_list: List[Token] = []
        self.stop_words: set = set(nltk.corpus.stopwords.words('english') + list(string.punctuation))
        self.case_sensitive = case_sensitive


    def add_document(self, doc_idx, doc):
        for token_idx, token in enumerate(doc):
            if len(self.posting_list) == 0:
                self.posting_list.append(Token(token))
                self.posting_list[0].docs.append({'doc_idx': doc_idx, 'indexes': [token_idx]})
                continue
            i = 0
            while i < len(self.posting_list) and token > self.posting_list[i].word:
                i += 1
            if i == len(self.posting_list):
                self.posting_list.append(Token(token))
                # self.posting_list[i].post_idx.append(post_idx)
            elif token != self.posting_list[i].word:
                self.posting_list.insert(i, Token(token))

            if doc_idx not in [elem['doc_idx'] for elem in self.posting_list[i].docs]:
                self.posting_list[i].docs.append({'doc_idx': doc_idx, 'indexes': [token_idx]})
            else:
                self.posting_list[i].docs[-1]['indexes'].append(token_idx)

    def create_posting_list(self):
        """
        calling this function, will create posting list of all occurred words cross all documents. in this function, we
        loop over all documents, then inside this loop, we loop over all the tokens that are in the current document.
        the we check if the length of posting_list is zero, then we add this token as first word. else if the length of
        posting_list is more than 0, we find the correct index of the token in posting_list alphabetically. then we check
        if this token, has been already in posting_list, we just add the current document index in tokens.docs, else, we
        add this token in the posting_list, then add the current document index.
            :parameter
                None
            :return
                None
        :return:
        """
        for doc_idx, doc in enumerate(self.documents):
            self.add_document(doc_idx=doc_idx, doc=doc)

    def get_token_index(self, x):
        """
        this function find index of a word in posting list using binary search algorithm.
            :parameter
                x:str
                    the word you want to find its index
            :return
                int: index of the word in posting_list
        """
        low = 0
        high = len(self.posting_list) - 1
        mid = 0
        while low <= high:
            mid = (high + low) // 2
            if self.posting_list[mid].word < x:
                low = mid + 1
            elif self.posting_list[mid].word > x:
                high = mid - 1
            else:
                return mid
        return -1

    def get_token(self, token):
        """
        This function will return the token object that contains docs informations. if the given token is not in the
        posting_list, it return the spell corrected token.
        :param token:
            token you want to fetch it from posting list
        :return:
            return the instance of token from posting list
        """
        p = self.get_token_index(token)
        if p == -1:
            null_token = Token('token')
            null_token.docs = []
            return null_token
        return self.posting_list[p]




## querying
At last we complete our IR system by implementing a QueryProcessor class. This class will use the InvertedIndex class we defined before.

In [5]:
class QueryProcessor:
    """
    In this class, I implement an information retrieval system which can search a query among documents.
    ...
    Attributes:
    -----------
    indexing_model: InvertedIndex
        an instance of InvertedIndex class which has been created by indexing documents.
    Methods
    -------
    Methods defined here:
        get_word_docs(self, word: str):
                this simple function gets a token and will return all index of documents that this token is appeared in.
                :param word:
                    a word that you want to search.
                :return:
                    list of indexes of documents.
        intersect(self, first_word, second_word):
                this function get two words, and find documents that both of these word has been occurred.
                :parameter
                first_word: str
                    first word you want to search
                second_word: str
                    second word you want to search
                :return
                    list of indexes of documents.
            union(self, first_word, second_word):
                this function get two words, and find documents that either each of these word has been occurred.
                :parameter
                first_word: str
                    first word you want to search
                second_word: str
                    second word you want to search
                :return
                    list of indexes of documents.
            not_in(self, word):
                this function get one word, and find documents that this word has been not occurred.
                :parameter
                word: str
                    the word you want to search
                :return
                    list of indexes of documents.
            near(self, first_word, second_word, length):
                this function get two words, and find documents that either each of these word has been occurred near by at most 3 words on left or right.
                :parameter
                first_word: str
                    first word you want to search
                second_word: str
                    second word you want to search
                :return
                    list of indexes of documents.
            search(self, query):
                this function get a query and recognize what kind of query is; then search the query.
                :parameter
                    query: str
                        the query that user wants to search
                :return
                    print list of indexes of documents in a pretty way.
    """
    def __init__(self, indexing_model):
        self.indexing_model = indexing_model

    def get_word_docs(self, word):
        t = self.indexing_model.get_token(word)
        result = set()
        for doc in t.docs:
            result.add(doc['doc_idx'])
        return result

    def intersect(self, first_word, second_word):
        docs1 = self.get_word_docs(first_word)
        docs2 = self.get_word_docs(second_word)
        return list(docs1 & docs2)

    def union(self, first_word, second_word):
        docs1 = self.get_word_docs(first_word)
        docs2 = self.get_word_docs(second_word)
        return list(docs1 | docs2)

    def not_in(self, word):
        all_docs = set(range(len(self.indexing_model.documents)))
        word_docs = self.get_word_docs(word)
        return list(all_docs - word_docs)

    def near(self, first_word, second_word, distance):
        result = set()
        t1 = self.indexing_model.get_token(first_word)
        t2 = self.indexing_model.get_token(second_word)
        for t in (t1, t2):
            for doc in t.docs:
                for idx in doc['indexes']:
                    if second_word in self.indexing_model.documents[doc["doc_idx"]][idx + 1:idx + 1 + distance]:
                        result.add(doc['doc_idx'])
        return list(result)

    def search(self, query):
        query_parts = query.lower().split()
        if 'and' in query_parts:
            return self.intersect(query_parts[0], query_parts[2])
        elif 'or' in query_parts:
            return self.union(query_parts[0], query_parts[2])
        elif 'not' in query_parts:
            return self.not_in(query_parts[1])
        elif 'near' in query:
            distance = int(query_parts[1].split('/')[1])
            return self.near(query_parts[0], query_parts[2], distance)
        else:
            return list(self.get_word_docs(query_parts[0]))


# Test
Now let's test our system using samples

### read documents
First let's define a documetns_test list.

In [6]:
# Sample documents:
documents_test = [
    'This is a simple example document. It contains several words. The words should be processed and indexed.',
    'Another example document with different content. Document indexing is important for retrieval.',
    'Another example document to test Boolean search capabilities. This document contains relevant content.']

### Build our system
Now let's build our system and call initial functions:

In [7]:
# Preprocess documents
preprocessed_documents = preprocess_documents(documents_test)

# Initialize the Information Retrieval System with the preprocessed documents
inverted_index = InvertedIndex(preprocessed_documents)

In [8]:
print('**** ir_system documents:\n', inverted_index.documents)

**** ir_system documents:
 [['simple', 'example', 'document', '.', 'contains', 'several', 'words', '.', 'words', 'processed', 'indexed', '.'], ['another', 'example', 'document', 'different', 'content', '.', 'document', 'indexing', 'important', 'retrieval', '.'], ['another', 'example', 'document', 'test', 'boolean', 'search', 'capabilities', '.', 'document', 'contains', 'relevant', 'content', '.']]


In [9]:
# Create posting lists
inverted_index.create_posting_list()

# Let's see the posting list
inverted_index.posting_list

[.,
 another,
 boolean,
 capabilities,
 contains,
 content,
 different,
 document,
 example,
 important,
 indexed,
 indexing,
 processed,
 relevant,
 retrieval,
 search,
 several,
 simple,
 test,
 words]

In [10]:
print('**** ir_system posting list:\n', inverted_index.posting_list)

**** ir_system posting list:
 [., another, boolean, capabilities, contains, content, different, document, example, important, indexed, indexing, processed, relevant, retrieval, search, several, simple, test, words]


### Testing
Now we can test and query to our system. Note that the documents are 0-based indexing:

In [11]:
ir_system = QueryProcessor(inverted_index)
# Now we test our sample
print(ir_system.search('example and content'))
print(ir_system.search('example or content'))
print(ir_system.search('not example'))
print(ir_system.search('example near/3 content'))


[1, 2]
[0, 1, 2]
[]
[1]


### Let's test it on our dataset file too. NOTE that the documents should be provided in the same directory:

In [12]:
dataset_path = '../dataset/raw'
documents_test = []
for filename in os.listdir(dataset_path):
    filepath = os.path.join(dataset_path, filename)
    if filepath.endswith('.txt'):
        print(filename)
        # Use encoding cp1252 for test cases if an error raised
        documents_test.append(open(filepath, encoding='cp1252').read())

Happy and Unhappy Renters.txt
A Festival of Books.txt
A Murder-Suicide.txt
Jerry Decided To Buy a Gun.txt
Better To Be Unlucky.txt
Pulling Out Nine Tons of Trash.txt
Food Fight Erupted in Prison.txt
Crazy Housing Prices.txt
Trees Are a Threat.txt
Man Injured at Fast Food Place.txt
Freeway Chase Ends at Newsstand.txt
Cloning Pets.txt
Sara Went Shopping.txt
Gasoline Prices Hit Record High.txt
Rentals at the Oceanside Community.txt


### Build our system
Now let's build our system and call initial functions:

In [13]:
# Preprocess documents
preprocessed_documents = preprocess_documents(documents_test)

# Initialize the Information Retrieval System with the preprocessed documents
inverted_index = InvertedIndex(preprocessed_documents)

In [14]:
print('**** ir_system documents:\n', inverted_index.documents)

**** ir_system documents:
 [['samantha', ',', 'like', 'many', 'renters', ',', 'tired', 'renting', '.', 'one', 'reason', 'annual', 'rent', 'goes', 'like', 'clockwork', '.', 'every', 'year', 'landlord', 'raises', 'rent', 'five', 'percent', '.', 'another', 'reason', 'neighbors', '.', '“', 'new', 'neighbors', 'always', 'seem', 'inconsiderate', 'ones', 'moved', ',', '”', 'said', '.', '“', 'first', 'neighbor', 'door-slammer', ';', 'always', 'knew', 'came', 'home', 'left', 'home', '.', 'moved', ',', 'saxophonist', 'moved', '.', 'saxophonist', '!', 'practiced', 'two', 'hours', 'day', '.', 'saturday', 'friends', 'would', 'come', '’', 'get', 'listen', 'whole', 'band', '.', 'called', 'police', ',', 'said', 'saxophone', 'playing', 'permitted', 'apartments', 'four', 'hours', 'day', ',', 'saxophone', 'playing', 'job-related', '.', 'told', 'lucky', 'guy', 'playing', 'two', 'hours', 'day', '!', '”', 'many', 'unhappy', 'renters', ',', 'also', 'happy', 'renters', '.', '“', '’', 'lucky', 'whole', 'life',

In [15]:
# Create posting lists
inverted_index.create_posting_list()

# Let's see the posting list
inverted_index.posting_list

[!,
 $,
 (,
 ),
 ,,
 .,
 .38,
 1,
 1,000,
 10,
 1000,
 1000x,
 100x,
 10x,
 11,
 110,000,
 12,
 120,
 120,000,
 15,
 15-percent,
 150,
 1992.,
 1993,
 1995,
 1x,
 2,000,
 2.09,
 2.14,
 2.22,
 20,
 20,000,
 200,
 230,000,
 24-year-old,
 25,
 25,000,
 25,000.,
 280,
 29.95,
 3,
 3.50,
 30,
 300,
 3037,
 39.95,
 4,000,
 4,000.residents,
 5,
 50,
 50,000,
 500,
 500x,
 50th,
 50x,
 510,000,
 6,000,
 60,
 65-year-old,
 7,
 70,000,
 70-year-old,
 74-year-old,
 75,
 75,000,
 76,
 79-year-old,
 87,
 9,
 9,000,
 90,
 99,
 9:00,
 :,
 ;,
 ?,
 a.m.,
 able,
 abuse,
 accidentally,
 according,
 acres,
 actually,
 administration,
 afghan,
 agency,
 ago,
 aid,
 alan,
 allen,
 allotments,
 allow,
 allowing,
 almost,
 alone,
 along,
 already,
 also,
 altadena,
 although,
 always,
 ambulance,
 american,
 ammunition,
 among,
 amount,
 amounts,
 amputated,
 angelenos,
 angeles,
 anniversary,
 annual,
 another,
 antennas,
 anyone,
 anything,
 apartment,
 apartments,
 apparent,
 appears,
 appliances,
 apply,


In [16]:
print('**** ir_system posting list:\n', inverted_index.posting_list)

**** ir_system posting list:
 [!, $, (, ), ,, ., .38, 1, 1,000, 10, 1000, 1000x, 100x, 10x, 11, 110,000, 12, 120, 120,000, 15, 15-percent, 150, 1992., 1993, 1995, 1x, 2,000, 2.09, 2.14, 2.22, 20, 20,000, 200, 230,000, 24-year-old, 25, 25,000, 25,000., 280, 29.95, 3, 3.50, 30, 300, 3037, 39.95, 4,000, 4,000.residents, 5, 50, 50,000, 500, 500x, 50th, 50x, 510,000, 6,000, 60, 65-year-old, 7, 70,000, 70-year-old, 74-year-old, 75, 75,000, 76, 79-year-old, 87, 9, 9,000, 90, 99, 9:00, :, ;, ?, a.m., able, abuse, accidentally, according, acres, actually, administration, afghan, agency, ago, aid, alan, allen, allotments, allow, allowing, almost, alone, along, already, also, altadena, although, always, ambulance, american, ammunition, among, amount, amounts, amputated, angelenos, angeles, anniversary, annual, another, antennas, anyone, anything, apartment, apartments, apparent, appears, appliances, apply, approved, april, arcadia, area, arizona, around, arrived, asked, asking, assisted, attendan

In [30]:
ir_system = QueryProcessor(inverted_index)
# Now we test our sample
print(ir_system.search('buy and car'))
print(ir_system.search('fire or water'))
print(ir_system.search('not example'))
print(ir_system.search('sports near/3 car'))

[12, 5]
[8, 10, 5]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
[4]
